# Scripts to fill SMPC 2019 abstract book, converting databases of presentations in to LaTex code

In [1]:
import collections
import os
import shutil
import io  
import sys  
import urllib
import csv
from pylatexenc.latexencode import utf8tolatex
from pylatexenc.latex2text import LatexNodes2Text

from time import strptime
from datetime import datetime
from datetime import date


from dumptruck import DumpTruck
from collections import OrderedDict
from collections import Counter
from collections import deque
from os import listdir, chdir,makedirs

# Functions defining LaTeX formating for Presentations for Abstracts book

In [2]:
# function to index affiliations per submission, 
# input list with entry per author
# output unique list of affiliations and matching index per author
#
# note: this only allows for single affilitions per author. 
#       too much work to redefine combos of institutions and collab centres per author
def InstSort(InstitutionList):
    #find unique Institutions
    InstList = []
    for inst in InstitutionList:
        yesAdd = 1
        for est in InstList:
            if inst == est:
                yesAdd = 0
        if yesAdd==1:
            InstList.append(inst)
    
    InstIndex = []
    for inst in InstitutionList:
        for k in range(len(InstList)):
            if InstList[k] == inst:
                InstIndex.append(k+1)
    
    InstitutionsSorted = [InstIndex, InstList]
    return InstitutionsSorted

In [3]:
# painfully specific fixes for characters that need to be recoded for latex output

def cleanLaTeX(textstring):
    a = utf8tolatex(textstring)
    a = a.replace("ɔ".decode('utf-8'), "\\textipa{O}")
    a = a.replace("ɛ".decode('utf-8'), "\\textipa{3}")
    a = a.replace("x̄".decode('utf-8'), "$\\bar{x}$")
    a = a.replace("𝛽".decode('utf-8'), "$\\beta$")
    a = a.replace("↔".decode('utf-8'), "$\leftrightarrow$")
    a = a.replace("♭".decode('utf-8'), "$\\flat$")
    a = a.replace("| אוניברסיטת תל-אביב".decode('utf-8'), "") # apologies but working in R2L language capacity into latex caused too many package conflicts to solve by publication time.
    a = a.replace("^3-^2-^1".decode('utf-8'), "\^3-\^2-\^1")
    a = a.replace("^4-{\#}^4-^5".decode('utf-8'), "\^4-{\#}\^4-\^5")
    a = a.replace("ṭ".decode('utf-8'), "\d{t}").encode('utf-8') 
    return a

In [4]:
# function producing latex code for a talk entry, including abstract
def talkTexAbstract(talk):
    tDetails = '\n\\vspace{5mm}\n'
    tDetails +='\\begin{tabular}{AB}\n'
    sessionTitle = ('\\textbf{\large %s-%i} & \\textbf{\large %s}\\\\\n' % 
                    (talk['SessionCode'],talk['TalkSessionNumber'], cleanLaTeX(talk['TalkTitle'])))
    tDetails+=sessionTitle

    Is = InstSort(talk['Institutions'])
    InstIndex = Is[0]
    InstList = Is[1]
    
    authors =str(talk['Time'])+ '& {\small '
    for i in range(len(talk['Authors'])):
        authors += cleanLaTeX(talk['Authors'][i]) # Author
        authors += '$^' + str(InstIndex[i]) + '$'# Institution affiliations
        authors += '\index{'+ cleanLaTeX(talk['AuthorIndex'][i]) +'}' # index
        if i < len(talk['Authors'])-1:
            authors += ', '
    authors += '}\\\\\n'

    instit =  talk['Room']+' & {\scriptsize {\itshape{$^1$' + cleanLaTeX(Is[1][0])
    for i in range(len(Is[1])-1):
        instit += ', $^' + str(i+2) + '$' + cleanLaTeX(Is[1][i+1])
    instit += '}}}\\\\\n'
    
    tDetails += authors + instit
    tDetails += '\n\end{tabular}\n\n'

    abstract = '\\noindent ' + cleanLaTeX(talk['Abstract']) + '\n\n'
    cont = ("Contact: \\textit{%s}\n\n" % (cleanLaTeX(talk['Contact'])))
    subjects = '\\textbf{Subjects:} '
    for k in talk['Subjects']:
        subjects += cleanLaTeX(k) + ', '
    subjects = subjects[:-2] + '\n\n'

    texTalk = tDetails + abstract + cont + subjects
    
    return texTalk

In [5]:
# function to produce acceptable LaTeX mark up for symposium entries (no times)
def SympTexAbstract(talk):
    if talk['Status'] == 'Accept Symposium':
        tDetails = ''
        tDetails +='\\begin{tabular}{AB}\n'
        Is = InstSort(talk['Institutions'])
        InstIndex = Is[0]
        InstList = Is[1]

        authors =str(talk['Time'])+ '& {\small '
        for i in range(len(talk['Authors'])):
            authors += cleanLaTeX(talk['Authors'][i]) # Author
            authors += '$^' + str(InstIndex[i]) + '$'# Institution affiliations
            authors += '\index{'+ cleanLaTeX(talk['AuthorIndex'][i]) +'}' # index
            if i < len(talk['Authors'])-1:
                authors += ', '
        authors += '}\\\\\n'


        instit =  talk['Room']+' & {\scriptsize {\itshape{$^1$' + cleanLaTeX(Is[1][0])
        for i in range(len(Is[1])-1):
            instit += ', $^' + str(i+2) + '$' + cleanLaTeX(Is[1][i+1])
        instit += '}}}\\\\\n'

        tDetails += authors + instit
        tDetails += '\n\end{tabular}\n\n'
        
        abstract = '\\noindent ' + cleanLaTeX(talk['Abstract']) + '\n\n'
        cont = ("Contact: \\textit{%s}\n\n" % cleanLaTeX((talk['Contact'])))
        texTalk = tDetails + abstract+cont
    else:
        tDetails = '\n\\vspace{5mm}\n'
        tDetails +='\\begin{tabular}{AB}\n'
        sessionTitle = ('\\textbf{\large %s-%i} & \\textbf{\large %s}\\\\\n' % 
                        (talk['SessionCode'],talk['TalkSessionNumber'], cleanLaTeX(talk['TalkTitle'])))
        tDetails+=sessionTitle

        Is = InstSort(talk['Institutions'])
        InstIndex = Is[0]
        InstList = Is[1]

        authors =' & {\small '
        for i in range(len(talk['Authors'])):
            authors += cleanLaTeX(talk['Authors'][i]) # Author
            authors += '$^' + str(InstIndex[i]) + '$'# Institution affiliations
            authors += '\index{'+ cleanLaTeX(talk['AuthorIndex'][i]) +'}' # index
            if i < len(talk['Authors'])-1:
                authors += ', '
        authors += '}\\\\\n'


        instit =  talk['Room']+' & {\scriptsize {\itshape{$^1$' + cleanLaTeX(Is[1][0])
        for i in range(len(Is[1])-1):
            instit += ', $^' + str(i+2) + '$' + cleanLaTeX(Is[1][i+1])
        instit += '}}}\\\\\n'

        tDetails += authors + instit
        tDetails += '\n\end{tabular}\n\n'

        abstract = '\\noindent ' + cleanLaTeX(talk['Abstract']) + '\n\n'

        texTalk = tDetails + abstract
    
    return texTalk

In [6]:
# function to lay out poster details of abstract book. 
# Was it a mistake to use a different field structure for posters and talks? yes and no.

def posterTexAbstract(poster):
    PosterCode = ("%s-%i" % (poster['PosterSession'],poster['PositionNumber']))
    SS = ("\subsection*{\\textit{%s} \hspace{0.5cm} %s}\n" % (PosterCode,cleanLaTeX(poster['PosterTitle'])))
    
    Is = InstSort(poster['Institutions'])
    InstIndex = Is[0]
    InstList = Is[1]
    
    authors = '\\noindent '
    for i in range(len(poster['Authors'])):
        authors += cleanLaTeX(poster['Authors'][i]) # Author
        authors += '$^' + str(InstIndex[i]) + '$'# Institution affiliations
        authors += '\index{'+ cleanLaTeX(poster['AuthorIndex'][i]) +'}' # index
        if i < len(poster['Authors'])-1:
            authors += ', '
    authors += '\n\n'
    
    instit =  '\\noindent {\scriptsize \\textit{$^1$' + cleanLaTeX(Is[1][0])
    for i in range(len(Is[1])-1):
        instit += ', $^' + str(i+2) + '$' + cleanLaTeX(Is[1][i+1])
    instit += '}}\\\\\n\\\\\n'

    abstract = '\\noindent ' + cleanLaTeX(poster['Abstract']) + '\n\n'
    cont = ("Contact: \\textit{%s}\n\n" % cleanLaTeX((poster['Contact'])))
    subjects = '\\textbf{Subjects:} '
    for k in poster['Subjects']:
        subjects += cleanLaTeX(k) + ', '
    subjects = subjects[:-2] + '\n\n'

    texPoster = SS + authors + instit + abstract + cont + subjects
    
    return texPoster


# Scripts translating day by day lists of presentations into abstract sections

In [7]:
chdir('./data')
dt = DumpTruck(dbname='Aug5_Talks.db')
ds = DumpTruck(dbname='Aug5_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Abstracts')

# OK now write to file
fileName = "Aug5_Talks_Abstract.tex"
f= open(fileName, 'w+')

f.write("\chapter*{August 5th Presentations}\\addcontentsline{toc}{chapter}{August 5th Presentations}\n\n")

#
i = 0
for k in range(len(data)/4):
    
    ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionSlot` = ' + str(k+1))
    #session slot title    
    slotTitle = ('\section*{%s \hspace{3cm} %s Sessions}' % (ss[0]['Times'],ss[0]['SessionCode'][0]))
    slotTitle += ('\\addcontentsline{toc}{section}{%s Sessions}\n\n' % (ss[0]['SessionCode'][0]))
    f.write(slotTitle)
    
    for s in ss:
        
        if s['Symposium'] == 0:
            sessionTitle = ('\subsection*{%s %s}' % (s['SessionCode'], utf8tolatex(s['SessionTitle'])))
            f.write(sessionTitle)
        else:
            sessionTitle = ('\subsection*{%s %s}' % (s['SessionCode'], utf8tolatex(s['SessionTitle'])))
            f.write(sessionTitle)

        ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))

        for talk in ts:
        #    print talk
            #print (str(talk['SubmissionNumber']) + ':' + talk['TalkTitle'])
            if talk['SubmissionNumber'] != 20: # cancelled talk
                if talk['Status'] == 'Accept Spoken':
                    r = talkTexAbstract(talk)
                else:
                    r = SympTexAbstract(talk)
                    #print  talk['Status']
                f.write(r)
            #print r
        i+=1
f.close() 
chdir('..')

In [8]:
chdir('./data')
dt = DumpTruck(dbname='Aug6_Talks.db')
ds = DumpTruck(dbname='Aug6_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Abstracts')


# OK now write to file
fileName = "Aug6_Talks_Abstract.tex"
f= open(fileName, 'w+')

f.write("\chapter*{August 6th Presentations}\\addcontentsline{toc}{chapter}{August 6th Presentations}\n\n")

i = 0
for k in range(len(data)/4):
    
    ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionSlot` = ' + str(k+1))
    #session slot title    
    slotTitle = ('\section*{%s \hspace{3cm} %s Sessions}' % (ss[0]['Times'],ss[0]['SessionCode'][0]))
    slotTitle += ('\\addcontentsline{toc}{section}{%s Sessions}\n\n' % (ss[0]['SessionCode'][0]))
    f.write(slotTitle)
    
    for s in ss:
        
        if s['Symposium'] == 0:
            sessionTitle = ('\subsection*{%s %s}' % (s['SessionCode'], utf8tolatex(s['SessionTitle'])))
            f.write(sessionTitle)
        else:
            sessionTitle = ('\subsection*{%s %s}' % (s['SessionCode'], utf8tolatex(s['SessionTitle'])))
            f.write(sessionTitle)

        ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))

        for talk in ts:
        #    print talk
            #print (str(talk['SubmissionNumber']) + ':' + talk['TalkTitle'])
            if talk['Status'] == 'Accept Spoken':
                r = talkTexAbstract(talk)
            else:
                r = SympTexAbstract(talk)
                #print  talk['Status']
            f.write(r)
            #print r
        i+=1
f.close()
chdir('..')

In [9]:
chdir('./data')
dt = DumpTruck(dbname='Aug7_Talks.db')
ds = DumpTruck(dbname='Aug7_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Abstracts')

# OK now write to file
fileName = "Aug7_Talks_Abstract.tex"
f= open(fileName, 'w+')

f.write("\chapter*{August 7th Presentations}\\addcontentsline{toc}{chapter}{August 7th Presentations}\n\n")

i = 0
for k in range(len(data)/4):
    
    ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionSlot` = ' + str(k+1))
    #session slot title    
    slotTitle = ('\section*{%s \hspace{3cm} %s Sessions}' % (ss[0]['Times'],ss[0]['SessionCode'][0]))
    slotTitle += ('\\addcontentsline{toc}{section}{%s Sessions}\n\n' % (ss[0]['SessionCode'][0]))
    f.write(slotTitle)
    
    for s in ss:
        
        if s['Symposium'] == 0:
            sessionTitle = ('\subsection*{%s %s}' % (s['SessionCode'], utf8tolatex(s['SessionTitle'])))
            f.write(sessionTitle)
        else:
            sessionTitle = ('\subsection*{%s %s}' % (s['SessionCode'], utf8tolatex(s['SessionTitle'])))
            f.write(sessionTitle)

        ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))

        for talk in ts:
        #    print talk
            #print (str(talk['SubmissionNumber']) + ':' + talk['TalkTitle'])
            if talk['Status'] == 'Accept Spoken':
                r = talkTexAbstract(talk)
            else:
                r = SympTexAbstract(talk)
                #print  talk['Status']
            f.write(r)
            #print r
        i+=1
f.close() 
chdir('..')

In [10]:
chdir('./data')
dt = DumpTruck(dbname='Aug6_Posters.db')
data = dt.dump()
chdir('..')
chdir('./Abstracts')

# OK now write to file
fileName = "Aug6_Poster_Abstract.tex"
f= open(fileName, 'w+')

f.write("\chapter*{August 6th Posters}\\addcontentsline{toc}{chapter}{August 6th Posters}\n\n")

for i in ['1','2']:
    ps = dt.execute('SELECT * FROM `dumptruck` WHERE `PosterSessionNumber` = ' + i)
    sessionTitle = ('\section*{%s in %s \hspace{0.5cm} %s}' % (ps[0]['Time'],ps[0]['Room'],ps[0]['PosterSession']))
    sessionTitle += ('\\addcontentsline{toc}{section}{%s Poster Session}\n\n' % (ps[0]['PosterSession']))
    f.write(sessionTitle)  
    
    for poster in ps:
    #    print poster
        #print (str(poster['SubmissionNumber']) + ':' + poster['PosterTitle'])
       
        r = posterTexAbstract(poster)
        #print r
        #if int(poster['SubmissionNumber']) != 154 and int(poster['SubmissionNumber']) != 201:
        f.write(r)
f.close() 
chdir('..')

In [11]:
chdir('./data')
dt = DumpTruck(dbname='Aug7_Posters.db')
data = dt.dump()
chdir('..')
chdir('./Abstracts')

# OK now write to file
fileName = "Aug7_Poster_Abstract.tex"
f= open(fileName, 'w+')

f.write("\chapter*{August 7th Posters}\\addcontentsline{toc}{chapter}{August 7th Posters}\n\n")

for i in ['3','4']:
    ps = dt.execute('SELECT * FROM `dumptruck` WHERE `PosterSessionNumber` = ' + i)
    sessionTitle = ('\section*{%s in %s \hspace{0.5cm} %s}' % (ps[0]['Time'],ps[0]['Room'],ps[0]['PosterSession']))
    sessionTitle += ('\\addcontentsline{toc}{section}{%s Poster Session}\n\n' % (ps[0]['PosterSession']))

    f.write(sessionTitle)  
    
    for poster in ps:
    #    print poster
        #print (str(poster['SubmissionNumber']) + ':' + poster['PosterTitle'])
        r = posterTexAbstract(poster)
        #print r
        f.write(r)
        #print r
f.close() 
chdir('..')

# Testing formating zone

In [32]:
b = talkTexAbstract(talk)
print b
fileName = "SubmissionDets.tex"
f= open(fileName, 'w+')
f.write(b)
f.write('\n\n')
f.close()


\vspace{5mm}
\begin{tabular}{AB}
\textbf{\large O4-4} & \textbf{\large Metaphorical Associations in Sound-Based Music as Mappings between Acoustical Properties and Semantic Domains}\\
4:30 PM& {\small Jason Noble$^1$\index{Noble, Jason}, {\'E}tienne Thoret$^1$\index{Thoret, {\'E}tienne}, Max Henry$^1$\index{Henry, Max}}\\
KC914 & {\scriptsize {\itshape{$^1$McGill University}}}\\

\end{tabular}

\noindent Contemporary {\textquotedblleft}sound-based{\textquotedblright} music presents rich opportunities for semantic study: in attenuating note-based organization, such music shifts the nexus of association from conventional musical units such as melodies onto properties such as timbre and texture. Metaphorical associations in such cases may consist in acoustical-to-semantic cross-domain mappings explicable through acoustical analysis. [P] In a perceptual experiment, 38 participants rated 40 excerpts of contemporary music along 23 semantic scales, e.g., rating the extent to which the music 